In [1]:
from nbfinder import NotebookFinder
import sys
sys.meta_path.append(NotebookFinder())
import numpy as np

In [1]:
def convert_to_2d(obj):
    orig_shape = obj.shape
    obj = np.transpose(obj,axes=(0,2,3,1))
    obj = obj.reshape(obj.shape[0]*obj.shape[1]*obj.shape[2], obj.shape[3])
    return obj, orig_shape
    

def convert_back_to_4d(obj, orig_shape):
    obj = obj.reshape(orig_shape[0], orig_shape[2],orig_shape[3], orig_shape[1])
    obj = np.transpose(obj, axes=(0,3,1,2))
    return obj

def softmax(ar):
    #numerically stable
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(ar - np.max(ar,axis=1,keepdims=True)) 

    scores = e_x / np.sum(e_x,axis=1, keepdims=True)
    return scores

def softmax4d(obj):

    obj, orig_shape = convert_to_2d(obj)
    probs = softmax(obj)
    probs_tens = convert_back_to_4d(probs,orig_shape)
    return probs_tens
    

def get_net_output(net):
    xywh_keys = ['xy_pred',
            'wh_pred']
    
    # conf_score and cls score both are raw logits, so we have to apply softmax
    conf_score = np.float32(net.blobs['obj_scores_copy'].data)
    conf_score = softmax4d(conf_score)
    cls_score = np.float32(net.blobs['class_scores_copy'].data)
    cls_score = softmax4d(cls_score)
    assert np.all(conf_score < 1.)
    xywh = [np.float32(net.blobs[k].data) for k in xywh_keys]
    net_out_blobs = xywh + [conf_score, cls_score]

    net_output = np.concatenate(tuple(net_out_blobs),axis=1)
    return net_output

In [3]:
# def get_everything(net_output,label):
#     conf_pred, cls_pred = get_obj_cls_scores_from_net_output(net_output)
#     conf_gt, cls_gt = get_obj_cls_scores_from_label(label)
#     pred_boxes = get_all_boxes(net_output,conf_pred,cls_pred)
#     gt_boxes = get_all_boxes(label,conf_gt,cls_gt)
#     return pred_boxes, gt_boxes

In [35]:
#! jupyter nbconvert --to script extract.ipynb

[NbConvertApp] Converting notebook extract.ipynb to script
[NbConvertApp] Writing 2630 bytes to extract.py
